In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

block_size = 3 # context length: how many charaters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [8]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [9]:
C = torch.randn((27, 2)) # C is lookup table for embeddings

In [11]:
C[5] # indexing the embedding table

tensor([ 0.5763, -1.0844])

In [13]:
C[torch.tensor([5, 6, 7, 7])] # indexing the embedding table with multiple indices

tensor([[ 0.5763, -1.0844],
        [ 0.8029,  0.5011],
        [-0.8627, -0.1206],
        [-0.8627, -0.1206]])

In [17]:
C[X] # get the embeddings for the multi dim

tensor([[[-2.6279, -0.6899],
         [-2.6279, -0.6899],
         [-2.6279, -0.6899]],

        [[-2.6279, -0.6899],
         [-2.6279, -0.6899],
         [ 0.5763, -1.0844]],

        [[-2.6279, -0.6899],
         [ 0.5763, -1.0844],
         [ 0.2437, -0.5343]],

        [[ 0.5763, -1.0844],
         [ 0.2437, -0.5343],
         [ 0.2437, -0.5343]],

        [[ 0.2437, -0.5343],
         [ 0.2437, -0.5343],
         [ 0.9836,  1.3922]],

        [[-2.6279, -0.6899],
         [-2.6279, -0.6899],
         [-2.6279, -0.6899]],

        [[-2.6279, -0.6899],
         [-2.6279, -0.6899],
         [-0.3576, -0.5920]],

        [[-2.6279, -0.6899],
         [-0.3576, -0.5920],
         [-1.5347,  0.3173]],

        [[-0.3576, -0.5920],
         [-1.5347,  0.3173],
         [ 1.1523, -1.0812]],

        [[-1.5347,  0.3173],
         [ 1.1523, -1.0812],
         [ 0.5383,  2.0330]],

        [[ 1.1523, -1.0812],
         [ 0.5383,  2.0330],
         [ 1.1523, -1.0812]],

        [[ 0.5383,  2

In [15]:
C[X].shape

torch.Size([32, 3, 2])

In [20]:
X[13, 2]

tensor(1)

In [23]:
C[X][13,2]

tensor([0.9836, 1.3922])

In [24]:
C[1]

tensor([0.9836, 1.3922])

In [25]:
emb = C[X] # embeddings
emb.shape

torch.Size([32, 3, 2])

In [26]:
# construct the hidden layer
W1 = torch.randn((6, 100)) # we have three inputs with 2 dim each 3*2 = 6, and 100 hidden neurons(This is up to us to decide)
b1 = torch.randn((100)) # biases for the hidden layer

In [27]:
emb @ W1 + b1 # wont work yet because of the shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [28]:
torch.cat(torch.unbind(emb, 1), 1).shape # One way to concatenate the embeddings shape but it will create a new storage

torch.Size([32, 6])

In [37]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # Another way to concatenate the embeddings shape uaing the view which is more efficient
h

tensor([[ 1.0000, -0.8243,  0.9974,  ..., -1.0000, -0.9830,  0.9829],
        [ 0.9996,  0.2747,  0.9996,  ..., -1.0000, -0.9765,  0.9996],
        [ 0.9955,  0.4283,  0.9960,  ..., -0.9998, -0.1696, -0.8686],
        ...,
        [ 0.9695, -0.8833,  0.9255,  ..., -0.9999,  0.0931,  0.5117],
        [-0.8003,  0.4855,  0.9999,  ..., -0.8724, -0.9740,  0.9968],
        [ 0.9378, -0.3617, -0.9694,  ..., -0.9992,  0.9964, -0.9893]])

In [38]:
h.shape

torch.Size([32, 100])

In [39]:
# construct the output/final layer
W2 = torch.randn((100, 27)) # we have 100 inputs and 27 possible characters
b2 = torch.randn((27)) # biases for the output layer